In [2]:
# load necessary packages
from huggingface_hub import login
from esm.models.esm3 import ESM3
from esm.sdk.api import ESM3InferenceClient, ESMProtein, GenerationConfig

/home/pkr/miniforge3/lib/python3.10/site-packages/esm/utils/structure/protein_structure.py:69: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)
/home/pkr/miniforge3/lib/python3.10/site-packages/esm/utils/structure/protein_structure.py:164: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)
/home/pkr/miniforge3/lib/python3.10/site-packages/esm/utils/structure/protein_structure.py:213: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)


In [3]:
import torch

In [4]:
# This will download the model weights and instantiate the model on your machine.
model: ESM3InferenceClient = ESM3.from_pretrained("esm3_sm_open_v1").to("cuda") # or "cpu"

/home/pkr/miniforge3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

/home/pkr/miniforge3/lib/python3.10/site-packages/esm/pretrained.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


In [5]:
# Generate a completion for a partial Carbonic Anhydrase (2vvb)
prompt = ""

# create ESMProtein object with the sequence
protein = ESMProtein(sequence=prompt)

# Generate the sequence, then the structure. This will iteratively unmask the sequence track.
protein = model.generate(protein, GenerationConfig(track="sequence", num_steps=8, temperature=0.7))
protein

100%|██████████| 8/8 [00:01<00:00,  7.67it/s]


ESMProteinError(error_msg='Cannot sample sequence when input has no masks.')

In [20]:
# We can show the predicted structure for the generated sequence.
protein = model.generate(protein, GenerationConfig(track="structure", num_steps=8))
protein

100%|██████████| 8/8 [00:00<00:00, 19.07it/s]


ESMProtein(sequence='ADASDSADSDAGDQATSLRILNNGHAFNVEFDDSQDKAVLKGGPLDGTYRLIQFHFHWGSLDGQGSEHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPPCTESVIWTVFKTPVEVSQSQVERSRSILTMDGPSGPNGRTGNFRPVQPLQNPLPGAVRX', secondary_structure=None, sasa=None, function_annotations=None, coordinates=tensor([[[  3.6075, -27.3798,   9.7120],
         [  2.5275, -27.2766,   8.7363],
         [  2.0837, -28.6558,   8.2601],
         ...,
         [     inf,      inf,      inf],
         [     inf,      inf,      inf],
         [     inf,      inf,      inf]],

        [[  2.2180, -28.5714,   8.1195],
         [  1.9468, -29.6865,   7.2183],
         [  1.1628, -29.2254,   5.9941],
         ...,
         [     inf,      inf,      inf],
         [     inf,      inf,      inf],
         [     inf,      inf,      inf]],

        [[  1.5749, -28.8617,   5.6425],
         [  1.3004, -28.9610,   4.2128],
         [ -0.1998, -29.0184,   3.9442],
         ...,
         [     inf

In [6]:
class OccupancyESM: 
    def __init__(self, model_name: str,
                 layers: list = [256],
                 activation = torch.nn.ReLU()
                 ):
        self.model = ESM3.from_pretrained(model_name).to("cuda")
        self.num_layers = len(layers)
        self.        


    def forward(self, input_sequence: str):
        x = ESMProtein(sequence=input_sequence)

   
        return output

ESM3(
  (encoder): EncodeInputs(
    (sequence_embed): Embedding(64, 1536)
    (plddt_projection): Linear(in_features=16, out_features=1536, bias=True)
    (structure_per_res_plddt_projection): Linear(in_features=16, out_features=1536, bias=True)
    (structure_tokens_embed): Embedding(4101, 1536)
    (ss8_embed): Embedding(11, 1536)
    (sasa_embed): Embedding(19, 1536)
    (function_embed): ModuleList(
      (0-7): 8 x Embedding(260, 192, padding_idx=0)
    )
    (residue_embed): EmbeddingBag(1478, 1536, mode='sum', padding_idx=0)
  )
  (transformer): TransformerStack(
    (blocks): ModuleList(
      (0): UnifiedTransformerBlock(
        (attn): MultiHeadAttention(
          (layernorm_qkv): Sequential(
            (0): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
            (1): Linear(in_features=1536, out_features=4608, bias=False)
          )
          (out_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (q_ln): LayerNorm((1536,), eps=1e-05, el